#Assignment 11:  Leaf Classification

In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import warnings
warnings.filterwarnings("ignore")

/kaggle/input/leaf-classification/train.csv.zip
/kaggle/input/leaf-classification/sample_submission.csv.zip
/kaggle/input/leaf-classification/images.zip
/kaggle/input/leaf-classification/test.csv.zip


In [24]:
# load data
train_data = pd.read_csv('../input/leaf-classification/train.csv.zip', index_col ='id')
test_data = pd.read_csv('../input/leaf-classification/test.csv.zip')

In [25]:
test_ids = test_data.id
test_data = test_data.drop(['id'], axis =1)

# Part 1: Data Preprocessing

In [26]:
train_data.head()

,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [27]:
# taking care of missing values
train_data.isnull().any().sum()

0

In [28]:
test_data.head()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,0.023438,...,0.006836,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711
1,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,0.023438,...,0.000000,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922
2,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,0.005859,...,0.128910,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000
3,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,0.001953,...,0.012695,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789
4,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,0.000000,...,0.000000,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812


In [29]:
test_data.isnull().any().sum()

0

In [30]:
# encoding catagorical
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 990 entries, 1 to 1584
Columns: 193 entries, species to texture64
dtypes: float64(192), object(1)
memory usage: 1.5+ MB


In [31]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Columns: 192 entries, margin1 to texture64
dtypes: float64(192)
memory usage: 891.1 KB


In [32]:
train_data.shape

(990, 193)

In [33]:
test_data.shape

(594, 192)

In [34]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
margin1,990.0,0.017412,0.019739,0.0,0.001953,0.009766,0.025391,0.087891
margin2,990.0,0.028539,0.038855,0.0,0.001953,0.011719,0.041016,0.205080
margin3,990.0,0.031988,0.025847,0.0,0.013672,0.025391,0.044922,0.156250
margin4,990.0,0.023280,0.028411,0.0,0.005859,0.013672,0.029297,0.169920
margin5,990.0,0.014264,0.018390,0.0,0.001953,0.007812,0.017578,0.111330
...,...,...,...,...,...,...,...,...
texture60,990.0,0.014017,0.060151,0.0,0.000000,0.000000,0.000000,0.578130
texture61,990.0,0.002688,0.011415,0.0,0.000000,0.000000,0.000000,0.151370
texture62,990.0,0.020291,0.039040,0.0,0.000000,0.003906,0.023438,0.375980
texture63,990.0,0.008989,0.013791,0.0,0.000000,0.002930,0.012695,0.086914


In [35]:
test_data.describe().T

,count,mean,std,min,25%,50%,75%,max
margin1,594.0,0.017562,0.019585,0.0,0.001953,0.009766,0.028809,0.085938
margin2,594.0,0.028425,0.038351,0.0,0.001953,0.010743,0.041016,0.189450
margin3,594.0,0.031858,0.025719,0.0,0.013672,0.023438,0.042969,0.167970
margin4,594.0,0.022556,0.028797,0.0,0.005859,0.013672,0.027344,0.164060
margin5,594.0,0.014527,0.018029,0.0,0.001953,0.007812,0.019531,0.093750
...,...,...,...,...,...,...,...,...
texture60,594.0,0.011217,0.052530,0.0,0.000000,0.000000,0.000000,0.606450
texture61,594.0,0.002617,0.011204,0.0,0.000000,0.000000,0.000000,0.123050
texture62,594.0,0.019975,0.034704,0.0,0.000000,0.003418,0.022461,0.247070
texture63,594.0,0.009389,0.013457,0.0,0.000000,0.002930,0.014648,0.086914


In [36]:
train_data['species'].nunique()

99

The target y is the only catagorical column

In [37]:
# IV and DV
x = train_data.drop('species',axis=1)
y = train_data['species']

In [38]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_fit = encoder.fit(train_data['species'])
y_label = y_fit.transform(train_data['species']) 
classes = list(y_fit.classes_) 
classes

['Acer_Capillipes',
 'Acer_Circinatum',
 'Acer_Mono',
 'Acer_Opalus',
 'Acer_Palmatum',
 'Acer_Pictum',
 'Acer_Platanoids',
 'Acer_Rubrum',
 'Acer_Rufinerve',
 'Acer_Saccharinum',
 'Alnus_Cordata',
 'Alnus_Maximowiczii',
 'Alnus_Rubra',
 'Alnus_Sieboldiana',
 'Alnus_Viridis',
 'Arundinaria_Simonii',
 'Betula_Austrosinensis',
 'Betula_Pendula',
 'Callicarpa_Bodinieri',
 'Castanea_Sativa',
 'Celtis_Koraiensis',
 'Cercis_Siliquastrum',
 'Cornus_Chinensis',
 'Cornus_Controversa',
 'Cornus_Macrophylla',
 'Cotinus_Coggygria',
 'Crataegus_Monogyna',
 'Cytisus_Battandieri',
 'Eucalyptus_Glaucescens',
 'Eucalyptus_Neglecta',
 'Eucalyptus_Urnigera',
 'Fagus_Sylvatica',
 'Ginkgo_Biloba',
 'Ilex_Aquifolium',
 'Ilex_Cornuta',
 'Liquidambar_Styraciflua',
 'Liriodendron_Tulipifera',
 'Lithocarpus_Cleistocarpus',
 'Lithocarpus_Edulis',
 'Magnolia_Heptapeta',
 'Magnolia_Salicifolia',
 'Morus_Nigra',
 'Olea_Europaea',
 'Phildelphus',
 'Populus_Adenopoda',
 'Populus_Grandidentata',
 'Populus_Nigra',
 'Pr

In [39]:
# splitting
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y_label, test_size = 0.2, random_state =1)

# Part 2: Building model

In [40]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 40)
classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=40)

In [41]:
from sklearn.metrics import classification_report
predictions = classifier.predict(x_test)
print (classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3
          12       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         2
          15       1.00      1.00      1.00         2
          16       1.00      1.00      1.00         2
          17       1.00    

In [42]:
final_predictions = classifier.predict_proba(test_data)

In [43]:
submission = pd.DataFrame(final_predictions, columns=classes)
submission.insert(0, 'id', test_ids)
submission.reset_index()

,index,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,0,4,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.000
1,1,7,0.000,0.000,0.100,0.050,0.000,0.0,0.000,0.025,...,0.0,0.025,0.000,0.0,0.0,0.0,0.000,0.025,0.0,0.000
2,2,9,0.000,0.525,0.025,0.000,0.075,0.0,0.000,0.000,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.025
3,3,12,0.000,0.000,0.000,0.025,0.000,0.0,0.025,0.000,...,0.0,0.000,0.025,0.0,0.0,0.0,0.050,0.000,0.0,0.050
4,4,13,0.025,0.000,0.000,0.000,0.000,0.0,0.000,0.025,...,0.0,0.000,0.050,0.0,0.0,0.0,0.025,0.000,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,589,1576,0.000,0.675,0.000,0.000,0.050,0.0,0.000,0.000,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.000
590,590,1577,0.000,0.025,0.000,0.000,0.000,0.0,0.000,0.025,...,0.0,0.000,0.025,0.0,0.0,0.0,0.000,0.000,0.0,0.025
591,591,1579,0.000,0.075,0.000,0.000,0.050,0.0,0.000,0.000,...,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.000,0.0,0.000
592,592,1580,0.000,0.000,0.025,0.000,0.000,0.0,0.000,0.000,...,0.1,0.050,0.000,0.0,0.0,0.0,0.000,0.025,0.0,0.000


In [44]:
submission.to_csv('result.csv', index = False)